# Decodifica di Hamming

Nelle telecomunicazioni i canali su cui viaggia l'informazione non sempre sono ideali. La richiesta che un canale sia privo di rumore, ovvero che un messaggio ricevuto non abbia subito delle modifiche durante il suo viaggio e che sia rimasto uguale quello inviato, è una richiesta molto forte.

### Ridondanza

A livello logico l'informazione è data da una sequenza di bit e gli errori consistono in delle modifiche casuali nella sequenza. L'idea è quindi di inviare dell'informazione in eccesso rispetto a quella da trasmettere e sfruttare questa ridondanza per individuare e correggere gli errori. Un protocollo di questo tipo potrebbe consistere nell'inviare $2k+1$ volte lo stesso bit e in fase di decodifica correggere l'errore usando il bit in maggioranza, ma sarebbe molto costoso in termini di *overhead*.

### Parità

Dato un messaggio di $N$ bit $b_1, \cdots, b_N$, un *<u>parity-check</u>* consiste nell'aggiunge un bit $c_p$ che consente di rilevare la presenza di un qualsiasi numero <u>dispari</u> di errori.

$$ \left( c_p + \sum_{i=1}^{N} b_i \right) \; \% \; 2 = 0 $$

Seppur abbia un costo minimo in termini di memoria, il parity check consente di rilevare solamente la presenza di un numero dispari e non permette di correggerli.

## Codici di Hamming

I codici di Hamming sono una famiglia di codici a blocchi lineari che sfruttano la ridondanza e il controllo di parità per la correzione automatica degli errori.

> **Codice lineare**: codice di correzione degli errori per cui ogni combinazione lineare tramite una somma bit a bit di parole di codice (*code-words*) valide è ancora una parola di codice.

> **Codice a blocchi**: codice di correzione degli errori che codifica i dati in blocchi.

I codici di Hamming sono famiglie di codici binari $(n, k)$ dove per ogni intero $r \ge 2$ c'è una code-word (un blocco) di lunghezza $n = 2^r - 1$ che contiene un messaggio di lunghezza $k = 2^r - r - 1$. Il blocco su cui opera Hamming è dunque dato da un messaggio di $k$ bit completato da un numero $r$ di bit di parità adibiti al controllo degli errori più un bit che per il momento non serve a nulla

$$ n = k + r + 1 \textrm{.}$$

Il *rate*, ovvero il rapporto tra la lunghezza del messaggio e del blocco, di un codice di Hamming è $R = k / n = 1 - r / (2^r - 1)$, che è il rate massimo per codici che hanno distanza di Hamming uguale a $3$.

> **Distanza di Hamming**: numero minimo di cambi di bit necessari per andare da una parola di codice valida a un'altra.

Il concetto di distanza è fondamentale poichè ci permette di distinguere due tipologie di codici di Hamming:

- *Single-error correcting* (**SEC**): algoritmo che non fa uso del bit supplementare che ha distanza di Hamming uguale a $3$.

    - Errore singolo: lo invididua e lo corregge.
    - Errore doppio: lo scambia per un errore singolo e lo corregge erroneamente.
    - Errore triplo: sequenza considerata valida dalla decodifica che quindi non apporta alcuna correzione.

- *Single-error correcting double-error detecting* (**SECDEC**): algoritmo che fa uso del bit supplementare per un ulteriore controllo di parità, consentendo di individuare (ma non correggere) fino a $2$ errori nella sequenza ed estendere la distanza a $4$.

    - Errore singolo: lo invididua e lo corregge.
    - Errore doppio: lo individua ma non lo corregge.
    - Errore triplo: lo scambia per un errore singolo e lo corregge erroneamente.
    - Errore quadruplo: sequenza considerata valida dalla decodifica che quindi non apporta alcuna correzione.

| Codice (n,k) | r (bit di parità) | n (lunghezza) | k (bit informativi) | Rate k/n |
|--------------|------------------|--------------|---------------------|----------|
| (3,1)        | 2                | 3            | 1                   | 0.333    |
| (7,4)        | 3                | 7            | 4                   | 0.571    |
| (15,11)      | 4                | 15           | 11                  | 0.733    |
| (31,26)      | 5                | 31           | 26                  | 0.839    |
| (63,57)      | 6                | 63           | 57                  | 0.905    |
| (127,120)    | 7                | 127          | 120                 | 0.945    |
| (255,247)    | 8                | 255          | 247                 | 0.969    |

### L'algoritmo

Nella logica della decodifica è fondamentale considerare le posizioni dei bit in binario, quindi gli indici della sequenza $1, 2, 3, 4, \cdots$ diventeranno $1, 10, 11, 100, \cdots$ ad esempio i bit di controllo si troveranno in posizione $1$, $10$, $100$, ecc.

L'idea dell'algoritmo di Hamming è la seguente:

1. Le posizioni detite ai bit di parità sono potenze di due: $1, 10, 100, 1000, \cdots$. La particolarità delle potenze di due in binario è che sono sequenze dove il bit all'estrema sinistra è $1$ e gli altri sono $0$.

2. Le altre posizioni, indicizzate da sequenze con due o più bit $1$, contengono i bit di informazione.

3. Ogni bit di informazione è incluso in un insieme unico di $2$ o più bit di parità, determinato dalla forma binaria della sua posizione.

    1. Il bit di parità $1$ copre tutte le posizioni di bit che hanno $1$ come bit che occupa la posizione più a destra in una stringa binaria: $1, 11, 101, 111, \cdots$ ($1, 3, 5, 7, \cdots$).

    2. Il bit di parità $2$ copre tutte le posizioni di bit che hanno $1$ come bit che occupa la seconda posizione da destra in una stringa binaria: $10- 110, 110-111, 1010-1011 \cdots$ ($2-3, 6-7, 10-11 \cdots$).

    3. Il bit di parità $4$ copre tutte le posizioni di bit che hanno $1$ come bit che occupa la terza posizione da destra in una stringa binaria: $100-111, 1100-1111, 10100-10111 \cdots$ ($4-7, 12-15, 20-23 \cdots$).

    4. In generale ogni bit di parità copre tutti i bit dove l'operazione bit a bit AND tra la posizione del bit di parità e del bit di informazione è non nulla.

>L'operazione bit a bit AND, indicata con $\&$, esegue un confronto tra due variabili dando come risultato una terza variabile che presenta un $1$ in quelle posizioni in cui entrambe le variabili di partenza presentano $1$ e uno $0$ in tutte le altre.

```python
a = 60          # 00111100
b = 240         # 11110000
c = a & b = 48  # 00110000
```

4. Le posizioni dei bit di controllo sono state scelte in maniera tale che l'operazione bit a bit XOR tra tutte le posizioni dei bit che contengono $1$ sia nulla.

> L'operazione bit a bit XOR indicata con $\; \^{} \;$, esegue un confronto tra due variabili dando come risultato una terza variabile che presenta un $1$ in quelle posizioni in cui le due variabili di partenza presentano valori diversi e uno $0$ in tutte le altre posizioni.

```python
a = 60          # 00111100
b = 240         # 11110000
c = a ^ b = 48  # 11001100
```

5. Se il ricevitore riceve una stringa che ha lo XOR degli indici nullo significa che il messaggio non è stato corrotto, altrimenti lo XOR indica l'indice del bit corrotto. Il risultato di questa operazione si chiama *sindrome*.

6. Al bit in posizione $0$, che fino ad ora è stato inutilizzato, affidiamo il compito di controllare la parità globale. Se la sindrome è non nulla e la parità globale è sbagliata c'è un singolo errore che il codice corregge, se la sindrome è non nulla e la parità globale è corretta c'è un doppio errore e si richiede l'invio del codice.

### Il rumore

Come possiamo modellizzare la presenza di rumore? Due comunicatori, che chiamiamo Alice e Bob, si scambiano una sequenza di bit su un canale. ${a_1, a_2, a_3, \cdots}$ sono i bit inviati da Alice, ${b_1, b_2, b_3, \cdots}$ sono i bit letti da Bob, dove $a_i,b_i \in \left\{0, 1\right\}$. Nel canale i bit non vengono dispersi (se entrano $N$ bit, escono $N$ bit) e il rumore è descritto da una probabilità non nulla $p_e$ che avvenga un *bit-flip* su un simbolo $a_i$ inviato da Alice

$$ p_e = P(a_i = 0 | b_i = 1) =  P(a_i = 1 | b_i = 0) \textrm{.}$$

Questa probabilità non dipende dal tempo o dal messaggio ma solo dal canale, che per essere "decente" deve avere $p_e \ll \frac{1}{2}$.

Nel settare $p_e$ bisogna rispettare l'ipotesi di un numero di errori per blocco $\#er \le 1$.

### Pseudocodice

```python
hamming_decoding(block)
    
```

### Correttezza

Per dimostrare la correttezza dell'algoritmo, affrontiamo il problema tramite un approccio matriciale, restringendoci al caso del codice di Hamming $(7,4)$. Introduciamo la matrice generatrice $\bold{G}$:

$$
\bold{G} = \left[ \begin{array}{c|c} \bold{I}_k & \bold{A} \end{array} \right] =
\begin{bmatrix}
1 & 0 & 0 & 0 & 1 & 1 & 0 \\
0 & 1 & 0 & 0 & 1 & 0 & 1 \\
0 & 0 & 1 & 0 & 0 & 1 & 1 \\
0 & 0 & 0 & 1 & 1 & 1 & 1
\end{bmatrix}_{4,7} \textrm{,}
$$

Applicando la trasposta di $\bold{G}$ a un vettore colonna $\bold{a}$ di $k$ bit di messaggio, e sommando modulo $2$, si ottiene una parola di codice $\bold{x}$ data dai bit di messaggio nelle prime posizioni e dagli $r$ bit di parità. Ad esempio per $\bold{a} = \left[1, 0, 1, 0 \right]$:

$$
\mathbf{x}^T = \mathbf{G}^T \mathbf{a}^T = 
\begin{bmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1 \\
1 & 1 & 0 & 1 \\
1 & 0 & 1 & 1 \\
0 & 1 & 1 & 1
\end{bmatrix} 
\begin{bmatrix}
1 \\
0 \\
1 \\
0
\end{bmatrix} = 
\begin{bmatrix}
1 \\
0 \\
1 \\
0 \\
1 \\
0 \\
1
\end{bmatrix} \textrm{.}
$$

Questo risultato rispecchia la struttura a blocchi della matrice generatrice, formata dall'identità $\bold{I}_k$ e dalla matrice $\bold{A}$ costruita elencando tutti i vettori colonna di altezza $k$ indipendenti a due a due. Se consideriamo nuovamente un messaggio $\bold{a}$ e definiamo il vettore $\bold{p} = \bold{a} \bold{A}$ come la lista dei bit di parità, la $j$-esima colonna di $\bold{A}$ dice quali bit di messaggio partecipano al $j$-esimo bit di parità. Ad esempio per il primo bit di parità:

$$
p_1 = \sum_{i=1}^{k} a_i A_{i,1} = \begin{bmatrix} 1 & 0 & 1 & 0 \end{bmatrix} \begin{bmatrix}
1 \\
1 \\
0 \\
1
\end{bmatrix} = 1 \cdot 1 + 0 \cdot 1 + 1 \cdot 0 + 0 \cdot 1 = 1
$$

Introduciamo la matrice di controllo della parità $\bold{H}$:

$$
\bold{H} = \left[ \begin{array}{c|c} -\bold{A}^T & \bold{I}_{n-k} \end{array} \right] =
\begin{bmatrix}
0 & 1 & 1 & 1 & 1 & 0 & 0 \\
1 & 0 & 1 & 1 & 0 & 1 & 0 \\
1 & 1 & 0 & 1 & 0 & 0 & 1
\end{bmatrix}_{3,7} \textrm{.}
$$

Per costruzione è tale per cui $\bold{H} \bold{G}^T = \bold{0}$, poiché $\bold{G}^T$ fornisce una parola di codice valida che applicata ad $\bold{H}$ deve dare un risultato nullo. Infatti, per la parola di codice $\bold{x}$ appena vista:

$$
\bold{H} \bold{x}^T = \begin{bmatrix}
0 & 1 & 1 & 1 & 1 & 0 & 0 \\
1 & 0 & 1 & 1 & 0 & 1 & 0 \\
1 & 1 & 0 & 1 & 0 & 0 & 1
\end{bmatrix} \begin{bmatrix}
1 \\
0 \\
1 \\
0 \\
1 \\
0 \\
1
\end{bmatrix} = \begin{bmatrix}
0 \\
0 \\
0 
\end{bmatrix}
$$

> **Correttezza**: dato un problema $\mathcal{P}$ e un insieme $\mathcal{I}$ delle istanze, $\mathcal{A}_{\mathcal{P}}$ è un algoritmo per la soluzione di $\mathcal{P}$ se e solo se $\mathcal{A}_{\mathcal{P}}(x) = y$ è una soluzione del problema $\forall x \in \mathcal{I}$.

Dimostrare la correttezza dell'algoritmo di Hamming significa dimostrare che per ogni parola di codice esso è in grado di individuare e correggere un errore, sotto ipotesi $\#er \le 1$. L'individuazione di un errore passa dal calcolo della sindrome

$$
\bold{s} = \bold{H} \bold{v}^T \textrm{,}
$$

dove $\bold{v}$ è una sequenza di bit che può essere valida o presentare un singolo errore.

1. <u>Nessun errore</u>: $\bold{v} = \bold{x}$ è una parola di codice, ed è quindi una sequenza valida poiché abbiamo visto la sua sindrome essere una sequenza di bit $0$

    $$
    \bold{s} = \bold{H} \bold{x}^T = \bold{0}
    $$

    Quindi l'algoritmo non introduce correzioni errate e la parola di codice resta valida.

2. <u>Errore singolo</u>: se avviene un errore singolo questo significa sommare modulo $2$ a $\bold{x}$ un vettore della base canonica $\bold{e}_i$. Calcoliamoci la sindrome:

    $$
    \bold{s} = \bold{H} \left( \bold{x} + \bold{e}_i \right)^T = \bold{H} \bold{x}^T + \bold{H} {\bold{e}_i}^T = \bold{H} {\bold{e}_i}^T = \bold{h}_i
    $$

    dove $\bold{h}_i$ è la colonna i-esima della matrice $\bold{H}$, i cui bit ci forniscono l'esatta posizione dell'errore. Inoltre, poiché le colonne di $\bold{H}$ sono tutte distinte tra loro, l'algoritmo identifica in maniera univoca la posizione di un errore e lo corregge.

## Implementazione

Messaggio che Alice vuole inviare a Bob

In [1]:
# importiamo le librerie necessarie
import random

In [2]:
# 'with' è un "gestore di contesto": garantisce che il file venga chiuso automaticamente non appena finiamo di leggere, evitando sprechi di memoria o errori di sistema
# questo comando apre il file "Alice.txt" in modalità lettura
with open("Alice.txt", "r") as file:
    # legge tutto il contenuto del file e rimuove eventuali spazi bianchi iniziali/finali
    input_str = file.read().strip() # stringa di input

print(input_str)

Ciao Bob!


Il messaggio viene prima convertito in codice ASCII, successivamente in codice binario

In [ ]:
# converte una stringa in una rappresentazione binaria, dove ogni carattere è rappresentato da 8 bit come lista di numeri
def string_to_binary(s):
    # per ogni carattere 'c' nella stringa, ottiene il codice ASCII con ord(c),
    # format(..., '08b') lo converte in binario con 8 bit con padding di zeri a sinistra, e crea una lista di interi
    # il ciclo più esterno (il primo) itera sui caratteri della stringa
    # il ciclo interno (il secondo) itera sui bit della stringa binaria di quel carattere e li converte in interi
    return [int(b) for c in s for b in format(ord(c), '08b')]

# lista che contiene il messaggio in rappresentazione binaria
input_bin = string_to_binary(input_str)

print(input_bin)

[0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1]


In [ ]:
k = 4 # lunghezza del messaggio in bit per ogni blocco

# funzione che calcola le posizioni dei bit di parità
# prende in input la lunghezza del messaggio
def parity_bits(k):
    r = 0
    while (2**r) < (k + r + 1):
        r += 1
    return [2**i for i in range(r)]

# funzione che calcola le posizioni dei bit riservati al messaggio
# prende in input la lunghezza del messaggio
def message_bits(k):
    r = len(parity_bits(k)) # numero dei bit di parità
    data_bits = []
    for i in range(1, k + r + 1):   # l'elemento con indice 0 è riservato al bit di parità globale
        # verifica se 'i' non è una potenza di 2 usando l'operatore bitwise AND (&)
        # se i è potenza di 2 (es. 1, 2, 4, 8), 'i & (i-1)' è sempre 0
        if (i & (i - 1)) != 0:
            data_bits.append(i)
    return data_bits

r = len(parity_bits(k))  # numero dei bit di parità
n = 1 + k + r  # lunghezza del blocco (incluso il bit di parità globale)

# lista che conterrà i blocchi di dati da inviare (bit di messaggio + bit di parità)
sent_data = []

# ciclo sulle divisioni del messaggio originario
for i in range(0, int(len(input_bin)/k)):
    message = input_bin[i*k:(i+1)*k]  # estraiamo il blocco di k bit dal messaggio originale
    block = [0] * (n) 
    # inseriamo nelle loro posizioni: bit di messaggio, bit di parità, bit di parità globale
    message_idx = 0
    for i in message_bits(k):
        block[i] = message[message_idx]
        message_idx += 1
    for i in parity_bits(k):
        parity = 0
        for j in range(1, n):
            if j & i:  # Se il risultato dell'AND è diverso da zero, significa che la posizione 'j' è controllata dal bit di parità corrente
                parity ^= block[j]  # il comando '^' è lo XOR per calcolare la parità
        block[i] = parity
    # calcolo del bit di parità globale
    overall_parity = sum(block) % 2
    block[0] = overall_parity
    sent_data.append(block)

# dividi la lista "input_bin" in sottoliste di 4 bit ciascuna
input_bin_div = [input_bin[i:i+4] for i in range(0, len(input_bin), k)]
print(input_bin_div)
print(sent_data)

[[0, 1, 0, 0], [0, 0, 1, 1], [0, 1, 1, 0], [1, 0, 0, 1], [0, 1, 1, 0], [0, 0, 0, 1], [0, 1, 1, 0], [1, 1, 1, 1], [0, 0, 1, 0], [0, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 1, 1, 0], [1, 1, 1, 1], [0, 1, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 0, 1]]
[[1, 1, 0, 0, 1, 1, 0, 0], [1, 1, 0, 0, 0, 0, 1, 1], [0, 1, 1, 0, 0, 1, 1, 0], [1, 0, 0, 1, 1, 0, 0, 1], [0, 1, 1, 0, 0, 1, 1, 0], [0, 1, 1, 0, 1, 0, 0, 1], [0, 1, 1, 0, 0, 1, 1, 0], [1, 1, 1, 1, 1, 1, 1, 1], [1, 0, 1, 0, 1, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 0, 0, 1, 1, 0, 0], [1, 0, 1, 0, 1, 0, 1, 0], [0, 1, 1, 0, 0, 1, 1, 0], [1, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 0, 0, 1, 1, 0], [1, 0, 1, 0, 1, 0, 1, 0], [1, 0, 1, 0, 1, 0, 1, 0], [0, 1, 1, 0, 1, 0, 0, 1]]


Inseriamo degli errori casuali per ogni blocco che viene inviato a Bob

In [19]:
# introduciamo degli errori casuali
p = 0.05  # probabilità di errore (bit flip)
# send = 1
sent_data_with_errors = []
for i in range(len(sent_data)):
    block_with_errors = sent_data[i].copy()
    for j in range(len(sent_data[i])):
        if random.random() < p:
            # Effettua il bit flip
            if sent_data[i][j] == 0:
                block_with_errors[j] = 1
            else:
                block_with_errors[j] = 0
    sent_data_with_errors.append(block_with_errors)

print(sent_data)
print(sent_data_with_errors)

[[1, 1, 0, 0, 1, 1, 0, 0], [1, 1, 0, 0, 0, 0, 1, 1], [0, 1, 1, 0, 0, 1, 1, 0], [1, 0, 0, 1, 1, 0, 0, 1], [0, 1, 1, 0, 0, 1, 1, 0], [0, 1, 1, 0, 1, 0, 0, 1], [0, 1, 1, 0, 0, 1, 1, 0], [1, 1, 1, 1, 1, 1, 1, 1], [1, 0, 1, 0, 1, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 0, 0, 1, 1, 0, 0], [1, 0, 1, 0, 1, 0, 1, 0], [0, 1, 1, 0, 0, 1, 1, 0], [1, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 0, 0, 1, 1, 0], [1, 0, 1, 0, 1, 0, 1, 0], [1, 0, 1, 0, 1, 0, 1, 0], [0, 1, 1, 0, 1, 0, 0, 1]]
[[1, 1, 0, 0, 1, 1, 0, 0], [1, 1, 0, 0, 0, 0, 1, 1], [0, 1, 1, 0, 0, 1, 1, 0], [1, 0, 0, 1, 1, 0, 0, 1], [0, 1, 1, 0, 0, 1, 1, 1], [0, 1, 1, 0, 1, 0, 0, 1], [0, 0, 1, 0, 0, 1, 1, 0], [1, 1, 1, 1, 1, 1, 0, 1], [1, 0, 1, 0, 1, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 0, 0, 1, 1, 0, 0], [1, 0, 1, 1, 1, 0, 1, 0], [0, 1, 1, 0, 1, 1, 1, 0], [1, 1, 1, 0, 1, 1, 1, 1], [0, 1, 1, 1, 0, 1, 1, 0], [1, 0, 1, 0, 1, 0, 1, 0], [1, 1, 1, 0, 1, 0, 1, 0], [0, 1, 1, 0, 0, 0, 0, 1]]


Proviamo a mandare a video il messaggio ricevuto da Bob con gli errori.

In [14]:
output_bin_with_errors = []

for i in range(len(sent_data_with_errors)):
    message = [sent_data_with_errors[i][j] for j in message_bits(k)]
    output_bin_with_errors.extend(message)
# converte una lista di bit (numeri 0 o 1) in una stringa alfanumerica
def binary_to_string(binary_list):
    # lista per raccogliere i caratteri convertiti
    result = []
    # itera sulla lista binaria a passi di 8 bit
    for i in range(0, len(binary_list), 8):
        # estrae un blocco di 8 bit
        byte = binary_list[i:i+8]
        # se il blocco ha esattamente 8 bit, lo converte
        if len(byte) == 8:
            # converte la lista in stringa binaria, poi in intero, poi in carattere ASCII
            byte_str = ''.join(str(b) for b in byte)
            char = chr(int(byte_str, 2))
            result.append(char)
    # unisce tutti i caratteri in una stringa
    return ''.join(result)

# output_str = binary_to_string(output_bin)
print(output_bin_with_errors)
output_str = binary_to_string(output_bin_with_errors)
print(output_str)

[0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1]
Kian Rob!


Correggiamo gli errori.

In [20]:
# funzione che calcola la sindrome del blocco ricevuto
def sindrome(block):
    s = 0
    for i in range(1, len(block)):
        if block[i] == 1:
            s ^= i
    return s

# funzione che calcola il bit di parità globale
def global_parity_check(block):
    return sum(block) % 2

# funzione che rileva e corregge gli errori nel blocco ricevuto
def correct_errors(block):
    s = sindrome(block) # calcola la sindrome del blocco ricevuto
    if s == 0:
        return block  # nessun errore rilevato
    else:
        block[s] ^= 1  # Corregge l'errore
        if global_parity_check(block) == 0:
            return block  # errore singolo corretto
        else:
            return 1  # errore multiplo rilevato, chiediamo di reinviare il blocco

print(sent_data)
print(sent_data_with_errors)

sent_data_corrected = []
for i in range(len(sent_data_with_errors)):
    send = 1
    while send == 1:
        send = correct_errors(sent_data_with_errors[i])
    sent_data_corrected.append(send)

print(sent_data_corrected)

[[1, 1, 0, 0, 1, 1, 0, 0], [1, 1, 0, 0, 0, 0, 1, 1], [0, 1, 1, 0, 0, 1, 1, 0], [1, 0, 0, 1, 1, 0, 0, 1], [0, 1, 1, 0, 0, 1, 1, 0], [0, 1, 1, 0, 1, 0, 0, 1], [0, 1, 1, 0, 0, 1, 1, 0], [1, 1, 1, 1, 1, 1, 1, 1], [1, 0, 1, 0, 1, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 0, 0, 1, 1, 0, 0], [1, 0, 1, 0, 1, 0, 1, 0], [0, 1, 1, 0, 0, 1, 1, 0], [1, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 0, 0, 1, 1, 0], [1, 0, 1, 0, 1, 0, 1, 0], [1, 0, 1, 0, 1, 0, 1, 0], [0, 1, 1, 0, 1, 0, 0, 1]]
[[1, 1, 0, 0, 1, 1, 0, 0], [1, 1, 0, 0, 0, 0, 1, 1], [0, 1, 1, 0, 0, 1, 1, 0], [1, 0, 0, 1, 1, 0, 0, 1], [0, 1, 1, 0, 0, 1, 1, 1], [0, 1, 1, 0, 1, 0, 0, 1], [0, 0, 1, 0, 0, 1, 1, 0], [1, 1, 1, 1, 1, 1, 0, 1], [1, 0, 1, 0, 1, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 0, 0, 1, 1, 0, 0], [1, 0, 1, 1, 1, 0, 1, 0], [0, 1, 1, 0, 1, 1, 1, 0], [1, 1, 1, 0, 1, 1, 1, 1], [0, 1, 1, 1, 0, 1, 1, 0], [1, 0, 1, 0, 1, 0, 1, 0], [1, 1, 1, 0, 1, 0, 1, 0], [0, 1, 1, 0, 0, 0, 0, 1]]
[[1, 1, 0, 0, 1, 1, 0, 0], [1, 1, 0, 0, 0, 0, 1, 1], [0, 1, 1,

In [21]:
output_bin_corrected = []

for i in range(len(sent_data_corrected)):
    message = [sent_data_corrected[i][j] for j in message_bits(k)]
    output_bin_corrected.extend(message)

output_str = binary_to_string(output_bin_corrected)
print(output_str)

Ciao Bob!
